In [1]:
import numpy as np
import pandas as pd

In [2]:
log = pd.read_csv('log_20100416_0.csv', engine='python')
pd.set_option("max_columns", 999)
pd.set_option("max_rows", 999)
log = log.fillna(0)
log.head()

,癤퓄og_date,big_log_id,log_id,actor,actor_account,target,target_account,worldnum,etc_str1,etc_str2,etc_str3,etc_num1,etc_num2,etc_num3,etc_num4,etc_num5,etc_num6,etc_num7,etc_num8,etc_num9,etc_num10,etc_num11,etc_num12,etc_num13,etc_num14,etc_num15,item_uid
0,2010-04-16 00:00:01.517,100,187,866194,20094023,0,0,220050001,0,0,0,45754.0,45978.0,1.0,50.0,1.0,4.0,26.0,224.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,327994181.0
1,2010-04-16 00:00:01.517,100,118,886535,20046800,0,0,220010000,0,0,0,1.0,6.0,1.0,3.0,329.0,520.0,60.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2010-04-16 00:00:01.517,200,260,808827,20092406,0,0,210060000,0,1,0,8146.0,2.0,169300006.0,0.0,50.0,1.0,11.0,22.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,398509520.0
3,2010-04-16 00:00:01.517,200,260,871174,20065359,0,0,220050000,0,1,0,6944.0,1.0,169300006.0,1.0,48.0,1.0,4.0,22.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,397490619.0
4,2010-04-16 00:00:01.517,100,160,819516,20036643,0,0,220050000,0,0,0,1.0,50.0,1.0,4.0,3818.0,1479.0,960.0,146070028.0,111.0,147840.0,401056.0,0.0,0.0,0.0,0.0,0.0


In [3]:
log["log_id"].dtypes

dtype('int64')

In [4]:
log_ids = [103, 104, 113, 118, 143, 202,719, 723, 151, 137, 138, 142, 145, 187, 188, 127, 601, 602, 603, 604, 605, 606,607, 608, 609, 610, 611, 612, 504, 505]
temp = log[log["log_id"].isin(log_ids)]

In [5]:
drop_data = ["big_log_id", "target", "target_account", "worldnum", "item_uid", "etc_str2", "etc_str3", "etc_num3", "etc_num4", "etc_num5", "etc_num6", "etc_num9", "etc_num10", "etc_num11", "etc_num13", "etc_num14", "etc_num15"  ]
temp.drop(drop_data, axis=1, inplace = True)
temp.head()

c:\users\백진우\appdata\local\programs\python\python37-32\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,癤퓄og_date,log_id,actor,actor_account,etc_str1,etc_num1,etc_num2,etc_num7,etc_num8,etc_num12
0,2010-04-16 00:00:01.517,187,866194,20094023,0,45754.0,45978.0,26.0,224.0,0.0
1,2010-04-16 00:00:01.517,118,886535,20046800,0,1.0,6.0,60.0,0.0,0.0
8,2010-04-16 00:00:01.517,202,872231,20041432,0,1.0,149.0,5.0,25.0,0.0
11,2010-04-16 00:00:01.517,143,872231,20041432,0,0.0,50.0,2250.0,3996.0,0.0
12,2010-04-16 00:00:01.517,202,830324,20042978,0,1.0,60.0,3.0,25.0,0.0


# Player information

login_day_count -> 103번 로그를 아이디별로 카운트

login_count -> 103번 로그 이용, 요일별로가 아니고 총 로그인 횟수

logout_day_count -> 104번 로그

play_time/per_day -> 104. etc_num7

avg_money -> 187, 188, etc_num8

ip_count -> 103번의 etc_str1

max_level -> 113번에서 etc_num2의 최고값


In [6]:
login_count = temp.loc[temp.log_id==103].groupby('actor').count()['log_id']
logout_count = temp.loc[temp.log_id==104].groupby('actor').count()['log_id']
play_time = temp.loc[temp.log_id==104].groupby('actor').sum()['etc_num7']
money_up = temp.loc[temp.log_id==187].groupby('actor').sum()['etc_num8']# - temp.loc[temp.log_id==188].groupby('actor').sum()['etc_num8']
money_down = temp.loc[temp.log_id==188].groupby('actor').sum()['etc_num8']
ip_count = temp.loc[temp.log_id==103].groupby('actor').count()['etc_str1']
max_level_a = temp.loc[temp.log_id==113].groupby('actor').max()['etc_num2']
max_level_b = temp.loc[temp.log_id==103].groupby('actor').max()['etc_num2']
if max_level_a > max_level_b:
    max_level = max_level_a
else:
    max_level = max_level_b

In [7]:
money_change = pd.merge(money_up,money_down,on = 'actor', how= 'outer').fillna(0)

In [8]:
money_change['change'] = money_change['etc_num8_x'] - money_change['etc_num8_y']

In [9]:
money_change.drop(['etc_num8_x','etc_num8_y'],axis=1,inplace=True)

In [10]:
tmp = pd.merge(money_change,play_time,on='actor',how='right')

In [11]:
tmp['result'] = tmp['change'] / tmp['etc_num7']

In [12]:
tmp.drop(['change','etc_num7'],axis=1,inplace=True)

In [13]:
avg_money = tmp

In [14]:
login_day_count = login_count.copy()

In [15]:
tmp2=pd.DataFrame(login_day_count)

In [16]:
tmp2['result'] = 1

In [17]:
tmp2.drop(['log_id'],axis=1,inplace=True)

In [18]:
login_day_count=tmp2

In [19]:
Player_information = pd.merge(login_count, logout_count, on='actor', how='outer')
Player_information = pd.merge(Player_information, login_day_count, on='actor', how='outer')
Player_information = pd.merge(Player_information, play_time, on='actor', how='outer')
Player_information = pd.merge(Player_information, avg_money, on='actor', how='outer')
Player_information = pd.merge(Player_information, ip_count, on='actor', how='outer')
Player_information = pd.merge(Player_information, max_level, on='actor', how='outer')

Player_information.columns = ["login_count", "logout_count",'login_day_count', "play_time" , "avg_money", "ip_count", "max_level"]
Player_information.fillna(0, inplace=True)
Player_information.head()

,login_count,logout_count,login_day_count,play_time,avg_money,ip_count,max_level
actor,,,,,,,
800040,1.0,0.0,1.0,0.0,0.000000,1.0,0.0
800235,1.0,1.0,1.0,416.0,745.694712,1.0,0.0
800354,1.0,0.0,1.0,0.0,0.000000,1.0,2.0
800485,2.0,2.0,1.0,159.0,-1132.075472,2.0,0.0
800515,2.0,2.0,1.0,12622.0,1.259071,2.0,0.0


# Player action

sit_count/per_day/ratio ->  118번 계정 UID별 횟수

exp_get_amount/per_day/ratio -> 143번 etc_num7

item_get_count/per_day/ratio -> 202, etc num1

exp_repair_count/per_day -> 148

money_get_count/per_day/ratio -> 187

abyss -> 719 변경후 랭킹순위 et_cnum8

use_portal_count/per_day -> 151

killed_by_pc -> 137

killed_by_npc -> 138

teleport_count/per_day -> 142

reborn_count -> 145

question_count -> 504(퀘스트완료)

In [20]:
sit_count = temp.loc[temp.log_id==118].groupby('actor').count()['log_id']
exp_get_amout = temp.loc[temp.log_id==143].groupby('actor').sum()['etc_num7']
item_get_count = temp.loc[temp.log_id==202].groupby('actor').sum()['etc_num1']
exp_repair_count = temp.loc[temp.log_id==148].groupby('actor').count()['log_id']
money_get_count = temp.loc[temp.log_id==187].groupby('actor').count()['log_id']
abyss = temp.loc[temp.log_id==719].groupby('actor').max()['etc_num8']
use_portal_count = temp.loc[temp.log_id==151].groupby('actor').count()['log_id']
killed_by_pc = temp.loc[temp.log_id==137].groupby('actor').count()['log_id']
killed_by_npc = temp.loc[temp.log_id==138].groupby('actor').count()['log_id']
teleport_count = temp.loc[temp.log_id==142].groupby('actor').count()['log_id']
reborn_count = temp.loc[temp.log_id==145].groupby('actor').count()['log_id']
question_count = temp.loc[temp.log_id==504].groupby('actor').count()['log_id']

In [21]:
Player_action = pd.merge(sit_count, exp_get_amout, on='actor', how='outer')
Player_action = pd.merge(Player_action, item_get_count, on='actor', how='outer')
Player_action = pd.merge(Player_action, exp_repair_count, on='actor', how='outer')
Player_action = pd.merge(Player_action, money_get_count, on='actor', how='outer')
Player_action = pd.merge(Player_action, abyss, on='actor', how='outer')
Player_action = pd.merge(Player_action, use_portal_count, on='actor', how='outer')
Player_action = pd.merge(Player_action, killed_by_pc, on='actor', how='outer')
Player_action = pd.merge(Player_action, killed_by_npc, on='actor', how='outer')
Player_action = pd.merge(Player_action, teleport_count, on='actor', how='outer')
Player_action = pd.merge(Player_action, reborn_count, on='actor', how='outer')
Player_action = pd.merge(Player_action, question_count, on='actor', how='outer')

Player_action.columns = ["sit_count", "exp_get_amout" ,"item_get_count", "exp_repair_count", "money_get_count", "abyss", "use_portal_count", "killed_by_pc", "killed_by_npc", "teleport_count", "reborn_count", "question_count"]
Player_action.fillna(0, inplace=True)
Player_action.head()

,sit_count,exp_get_amout,item_get_count,exp_repair_count,money_get_count,abyss,use_portal_count,killed_by_pc,killed_by_npc,teleport_count,reborn_count,question_count
actor,,,,,,,,,,,,
800017,7.0,749199.0,37.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
800040,1.0,13946.0,3.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
800044,3.0,874731.0,16.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
800061,1.0,1131278.0,7.0,0.0,17.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
800117,8.0,946213.0,29.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Group activity

average_party_time 127, etc_num 12 에다가 127의 횟수로 나누기

guild_act_count 601-612 다 더하기

guild_join_count 605


In [22]:
total_party_time = temp.loc[temp.log_id==127].groupby('actor').sum()['etc_num12']
guild_join_count = temp.loc[temp.log_id==605].groupby('actor').count()['log_id']

In [23]:
out_count = temp.loc[temp.log_id==127].groupby('actor').count()['log_id']

In [24]:
tmp3 = pd.merge(total_party_time, out_count,on='actor',how='outer')

In [25]:
tmp3['result'] = tmp3['etc_num12'] / tmp3['log_id']

In [26]:
tmp3.drop(['etc_num12','log_id'],axis=1,inplace=True)

In [27]:
average_party = tmp3

In [28]:
g1=temp.loc[temp.log_id==601].groupby('actor').count()['log_id']
g2=temp.loc[temp.log_id==602].groupby('actor').count()['log_id']
g3=temp.loc[temp.log_id==603].groupby('actor').count()['log_id']
g4=temp.loc[temp.log_id==604].groupby('actor').count()['log_id']
g5=temp.loc[temp.log_id==605].groupby('actor').count()['log_id']
g6=temp.loc[temp.log_id==606].groupby('actor').count()['log_id']
g7=temp.loc[temp.log_id==607].groupby('actor').count()['log_id']
g8=temp.loc[temp.log_id==608].groupby('actor').count()['log_id']
g9=temp.loc[temp.log_id==609].groupby('actor').count()['log_id']
g10=temp.loc[temp.log_id==610].groupby('actor').count()['log_id']
g11=temp.loc[temp.log_id==611].groupby('actor').count()['log_id']
g12=temp.loc[temp.log_id==612].groupby('actor').count()['log_id']

In [29]:
G = pd.merge(g1, g2, on='actor', how='outer')
G = pd.merge(G, g3, on='actor', how='outer')
G = pd.merge(G, g4, on='actor', how='outer')
G = pd.merge(G, g5, on='actor', how='outer')
G = pd.merge(G, g6, on='actor', how='outer')
G = pd.merge(G, g7, on='actor', how='outer')
G = pd.merge(G, g8, on='actor', how='outer')
G = pd.merge(G, g9, on='actor', how='outer')
G = pd.merge(G, g10, on='actor', how='outer')
G = pd.merge(G, g11, on='actor', how='outer')
G = pd.merge(G, g12, on='actor', how='outer')

In [30]:
G.fillna(0)

,log_id_x,log_id_y,log_id_x,log_id_y,log_id_x,log_id_y,log_id_x,log_id_y,log_id_x,log_id_y,log_id_x,log_id_y
actor,,,,,,,,,,,,
813738,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
835480,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
846031,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
848511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
887131,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
897539,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
831980,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
832382,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
863655,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
G['S'] = G.sum(axis=1)

In [35]:
Group_activity = pd.merge(total_party_time, guild_join_count, on='actor', how='outer')
Group_activity = pd.merge(Group_activity, average_party, on='actor', how='outer')
Group_activity.columns = ["total_party_time", "guild_join_count", "average_party_time"]
Group_activity.fillna(0, inplace=True)
Group_activity.head()

,total_party_time,guild_join_count,average_party_time
actor,,,
0,251954.0,0.0,4270.40678
800040,78.0,0.0,78.00000
801224,1970.0,0.0,1970.00000
801701,2194.0,0.0,2194.00000
803416,2545.0,0.0,1272.50000


# csv 파일로 만들기

In [197]:
Player_information.to_csv("Player_information.csv", encoding="utf8")
Player_action.to_csv("Player_action.csv", encoding="utf8")
Group_activity.to_csv("Group_activity.csv", encoding="utf8")